In [1]:
import subprocess
import cv2
from ultralytics import YOLO


In [5]:

model = YOLO("yolov9c.pt")


# Remplace VIDEO_ID par l'ID de la vidéo YouTube en direct
video_url = "https://www.youtube.com/watch?v=ByED80IKdIU"
command = ["yt-dlp", "-g", video_url]

# Exécuter la commande et capturer l'URL du flux
try:
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
    url_m3u8 = result.stdout.strip()

    print("URL du flux m3u8:", url_m3u8)

    # Lire le flux vidéo avec OpenCV
    cap = cv2.VideoCapture(url_m3u8)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Erreur : Impossible de lire le flux vidéo")
            break

        
        results = model(frame)
        
        for result in results:
            
            boxes = result.boxes  # Boîtes de délimitation
            if boxes is not None :
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Coordonnées des coins
                    
                    conf = box.conf[0]  # Confiance
                    
                    if conf >= 0.3 :
                        cls = box.cls[0]  # Classe de l'objet détecté

                        # Afficher les rectangles de détection et les étiquettes
                        label = f"{model.names[int(cls)]}: {conf:.2f}"
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow('Live Stream with YOLOv8 Detection', frame)
        # Quitter avec la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

except subprocess.CalledProcessError as e:
    print(f"Erreur lors de l'exécution de la commande : {e}")
    print(f"Code de retour : {e.returncode}")
    print(f"Message d'erreur : {e.stderr}")


URL du flux m3u8: https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1724296542/ei/_ljGZqb8DIH2xN8Pr9KUuAQ/ip/2a01:cb05:4c0:4100:d8d0:46b0:ae3f:f8ac/id/ByED80IKdIU.2/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr4---sn-25glenl6.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/spc/Mv1m9rTXojW5uPAXCZk3GDB3sFggkaBcHEvpj8Ly3ZIWZ1cAmNOOhyM/vprv/1/playlist_type/DVR/initcwndbps/1552500/mh/jK/mm/44/mn/sn-25glenl6/ms/lva/mv/m/mvi/4/pl/42/dover/11/pacing/0/keepalive/yes/mt/1724274278/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,spc,vprv,playlist_type/sig/AJfQdSswRQIgF62KBWcBxWxlY1shLlc9FfKlFPgEzlG9KoivrdO8pBcCIQCXG_9ut9sSJ6a5s9Bt0XYBih5MaS1ck4BjaQdif_QvTQ%3D%3D/lsparams/hls_chunk_host,initcwndbps,mh,mm,mn,ms,mv,mvi,pl/lsig/AGtxev0wRQIgE7IBjI9EjJF7cJFplQgQsl0QTXEuJX-Zng2KVsZcMgQCIQDn96-vqq7PIEK315VmxbyKoDgxhovE4IIN0EJ_rQkfeQ%3D%3D/playlist/i